# Lightweight component iteration (Py3)

In [ ]:
# Install Kubeflow Pipeline SDK
!pip3 install kfp --upgrade

## Iterate on a Python function in a component

In [ ]:
import kfp.components as components

def add(a: float, b: float) -> float:
    """This function docstring appears as the component description."""
    return a + b

# add() is embedded into the component, on `base_image`.
add_op = components.func_to_container_op(add, base_image='tensorflow/tensorflow:1.12.0-py3')

## Build a pipeline with the component under iteration inside

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Lightweight pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def add_pipeline(
    a = '12', b = '15', # Default value
):
    add_task = add_op(a, b)

In [ ]:
pipeline_func = add_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.tar.gz'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [ ]:
from kfp_experiment.rest import ApiException

EXPERIMENT_NAME = 'Lightweight iteration'

#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()

try:
    experiment = client.create_experiment(EXPERIMENT_NAME)
except ApiException as e:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
    
run_name = pipeline_func.__name__ + ' Py3 Run'
run_result = client.run_pipeline(
    experiment.id, run_name, pipeline_filename, {'a': 10, 'b': 20})